# Lecture 7

# Section 5: Using a professional cloud deployed database such as MS Azure SQL Server
This section has advanced (and optional) content. It will require quite a number of steps and configurations and is rather involving.

## Preparations for using an MS Azure SQL Database

- you need to register a free Azure SQL database [here](https://aka.ms/freedb)

On a Mac computer
- specifically you may need to install the ODBC driver, see [here](https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/install-microsoft-odbc-driver-sql-server-macos?view=sql-server-ver16) 
- see also the trouble shooting section on that page, this has been necessary for me

> On an Intel Mac I needed to create symbolic links to the driver via 
>```console
>sudo ln -s /usr/local/etc/odbcinst.ini /etc/odbcinst.ini
>sudo ln -s /usr/local/etc/odbc.ini /etc/odbc.ini
>```

> This **does not work on an ARM Mac** (M1, M2, etc. processors). Here I needed to create symbolic links to the driver at other places in the filesystem via 
>```console
>sudo ln -s /opt/homebrew/etc/odbcinst.ini /etc/odbcinst.ini
>sudo ln -s /opt/homebrew/etc/odbc.ini /etc/odbc.ini
>```

On a Windows computer
- I could not try it on a Windows machine
- It may work out of the box
- If not start with downloading and installing the ODBC driver [here](https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver16)

## For using this and other databases in Python some libraries are useful
- you may need to install _pyodbc_ with `pip install pyodbc` 
- and _sqlalchemy_ with `pip install sqlalchemy` 
- the cell below does this for you

In [ ]:
# potentially you need first to install a number of libraries
 
#these libraries help to connect to databases
%pip install -q pyodbc
%pip install -q sqlalchemy


### Dealing with passwords in python code

Here is an interesting read on [how to deal with passwords in python code](https://stackoverflow.com/questions/7014953/i-need-to-securely-store-a-username-and-password-in-python-what-are-my-options)

And here are mechanisms on [how to deal with such issues in (public) streamlit apps](https://docs.streamlit.io/library/advanced-features/secrets-management)

In [ ]:
# this imports my password file - I did not share this file with you but you've got the template
import passwd

In [ ]:
import pyodbc 
import pandas as pd

server = 'fcsdb.database.windows.net'
database = 'fcsdb'
username = 'fcsdb'
password = passwd.db_password   # here it reads **my** password from the file, 
                                # this will not work fou you, you need your own password
                                # from your own password file (see my template)
                                # and your own MS Azure account
driver= '{ODBC Driver 18 for SQL Server}'

The followign code should work without errors if you can connect to your Azure SQL database - even if the database is empty. If this cell does not work, try to fix the errors before you continue. Possible errors:
- you have not installed and/or linked the ODBC driver correctly
- you have not adjusted the Azure firewall rules accordingly (check in the Azure database portal)
- you have not entered the correct SQL credentials for your database
- jou have no database

In [ ]:
with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT TOP 3 name, collation_name FROM sys.databases")
        row = cursor.fetchone()
        while row:
            print (str(row[0]) + " " + str(row[1]))
            row = cursor.fetchone()

You can adapt the code above to run whatever SQL query on your database.

## Using the `Chinook` database on Azure SQL server
- first you need an _SQL Server version_ of the `chinook` database, you find all kinds of versions [here](https://github.com/lerocha/chinook-database) and the SQL Server version [here](https://github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_SqlServer.sql)
- for importing the `chinook` database to Azure SQL server database I installed _Azure Data Studio_ onto my local machine, you find it [here](https://aka.ms/azuredatastudio) 
- start and connect _Azure Data Studio_ with your Azure database
- run the `chinook` SQL script downloaded in the first step (there may be some errors, **it may be best to remove the `Drop database if it exists` and `Create database` sections right at the start of the sql script** because you want to create tables and tuples directly into your existing Azure database) 

### Setting up the database connection
Now we use a similar but a bit easier approach to the above to query our database.
First we need to import the quitered packages and setup our database settings. Again, the seetings here will not work for you, you need to put your credentials in here. 

In [ ]:
import urllib
from sqlalchemy import create_engine

# Adjust the database settings to your Azure database 
server = 'fcsdb.database.windows.net'
database = 'fcsdb'
username = 'fcsdb'
password = passwd.db_password
driver= '{ODBC Driver 18 for SQL Server}'

### Connecting to the database
Here we get a `DB` object, similar to the one from the SQLite database before

In [ ]:
conn = f"""Driver={driver};Server=tcp:{server},1433;Database={database};
Uid={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"""

params = urllib.parse.quote_plus(conn)
conn_str = 'mssql+pyodbc:///?autocommit=true&odbc_connect={}'.format(params)
DB = create_engine(conn_str, echo=False)

### Querying the database
In theory, we now can use exactly the same code for querying the Azure database as we used for the SQLite database. Unfortunately, the imported version uses (or whatever reason) slightly different table names, e.g., `Album` and `Artist` onstead of `Albums` and `Artists`. We could fix that but I did not do it here.

To get an overview on those table names we first query `sys.tables` for all the table names and the further on use those slightly different names in our original SQLite SQL queries. Here we go:

In [ ]:
# getting the table names
pd.read_sql('SELECT name FROM sys.tables', DB) 

In [ ]:
# getting the number of customers
pd.read_sql('SELECT count(*) FROM Customer', DB) 

In [ ]:
# getting all employees
pd.read_sql('SELECT * FROM employee', DB) 

In [ ]:
# Your code goes here:


### Using an Azure SQL server database in `streamlit`
You may use your database in streamlit as is, or you may also use some even simplers ways to interact with an SQL server database (and many other data sources) with `streamlit` means. The latter me be helpful if you want to deploy your app to the streamlit cloud.

Please find the documentation [here](https://docs.streamlit.io/knowledge-base/tutorials/databases/mssql)

When deploying your app with user input and dynamically created SQL queries to the internet, be aware of the threat of **sql injections**. [Here](https://realpython.com/prevent-python-sql-injection/) and in many other places you can read what an sql injection is and how to prevent them. This is an important but advanced topic.

In the context of using databases in `streamlit` but also for other approaches of **making data persistent beyond a run of a `streamlit` program or even beyond a `streamlit` session**, have a look at the advanced concepts of **[statefulness](https://docs.streamlit.io/library/advanced-features/session-state)** and **[caching](https://docs.streamlit.io/library/advanced-features/caching)**.

__Happy Coding!__